In [ ]:
import os
import numpy as np
# import working directory to check functions
os.chdir('/Users/Dominik/R-workspace/cecelia/inst')

# config
import py.config_utils as cfg

%load_ext autoreload
%autoreload 2

In [ ]:
base_dir = '/Volumes/Analysis_SSD/Dominik/cecelia/projects/wEsnzt/ANALYSIS/'
zero_dir = os.path.join(base_dir, '0/n22foC/')
#im_path = os.path.join(zero_dir, 'ccidImage.ome.zarr')
im_path = os.path.join(zero_dir, 'ccidRegistered.zarr')
version_num = 1
task_dir = os.path.join(base_dir, str(version_num), 'n22foC')

In [ ]:
base_dir = '/Volumes/Analysis_SSD/Dominik/cecelia/projects/Wq59Z0/ANALYSIS/'
zero_dir = os.path.join(base_dir, '0/f6YZeo/')
im_path = os.path.join(zero_dir, 'ccidImage.ome.zarr')
version_num = 1
task_dir = os.path.join(base_dir, str(version_num), 'f6YZeo')

In [ ]:
import py.ome_xml_utils as ome_xml_utils
from py.dim_utils import DimUtils
from py.label_props_utils import LabelPropsUtils
import py.zarr_utils as zarr_utils
import pandas as pd

In [ ]:
view = LabelPropsUtils(task_dir).label_props_view(value_name = 'default')

In [ ]:
view.change_channel_names(['YAP', 'FRC', 'TUM', 'DAP', 'AF'])

In [ ]:
view.col_names(data_type = 'vars')

In [ ]:
from py.napari_utils import NapariUtils

napari_utils = NapariUtils()
napari_utils.viewer = None
napari_utils.open_viewer()
napari_utils.task_dir = task_dir

In [ ]:
channel_names = [
    #'BCL2','CD10','Hoechst 33342','CD20','CD3','Collagen IV','CD23','BCL6','CD138','IgD','CD31','CD21','CD163','HLA-DR','CD11c','CD1c','CD25','FOXP3','CD8','CD69','PD-1','CD4','CD38','SPARC','CD68','IRF4','ICOS','Ki-67','CD35','CXCL13','TCR Va7.2','CD44','CD106','a-SMA','Lyve-1','DC-SIGN','CD45','CLEC9a AF488'
    'CD21','CD54','Hoechst','CD20','CD68','Glycophorin A','Fibrinogen','CD163','CD49a','CD138','CD15','HLA-DR','CD11c','Vimentin','CD4','CD31','Ki-67','CD3','CD8','SPARC','CD44','CD61','CD45','SMA','Lumican'
]

In [ ]:
im_res = 0.2840914

im_slices = (
    slice(None), slice(None), slice(None),
    slice(round(925 / im_res), round(980 / im_res), 1),
    slice(round(955 / im_res), round(1200 / im_res), 1)
)
stroma_slices = (
    slice(None), slice(None),
    slice(round(925 / im_res), round(980 / im_res), 1),
    slice(round(955 / im_res), round(1200 / im_res), 1)
)

In [ ]:
napari_utils.open_image(
    im_path,
    use_channel_axis = True, as_dask = True,
    visible = False, channel_names = channel_names,
    multiscales = 1, show_3D = True, slices = im_slices
)

In [ ]:
napari_utils.show_labels_all(
    show_labels = True,
    show_points = False,
    value_names = ['stroma']
)

In [ ]:
napari_utils.show_labels_all(
    show_labels = True,
    show_points = False,
    value_names = ['stroma.branch'],
    show_branching = True,
    slices = stroma_slices
)

In [ ]:
napari_utils.show_labels_all(
    show_labels = True,
    show_points = False,
    value_names = ['default'],
    label_suffixes = {'default': ['cyto', 'nuc']}
)

In [ ]:
im_data, _ = zarr_utils.open_zarr(im_path, as_dask = True)
omexml = ome_xml_utils.parse_meta(im_path)
dim_utils = DimUtils(omexml)
dim_utils.calc_image_dimensions(im_data[0].shape)

In [ ]:
branch_path = os.path.join(task_dir, 'labels', 'stroma.branch.zarr')
labels_path = os.path.join(task_dir, 'labels', 'default.zarr')

In [ ]:
# assign branching to cell labels
branch_data, _ = zarr_utils.open_labels_as_zarr(branch_path)
labels_data, _ = zarr_utils.open_labels_as_zarr(labels_path)

In [ ]:
# TODO this is not good
rgb = np.dstack(tuple([
    np.amax(skeleton_labels[stroma_slices] * np.isin(
    skeleton_labels[stroma_slices], paths_table.loc[paths_table['branch-type'] == x]['path-id'].values),
            axis = 0)
    for x in range(0, 3)
]))

In [ ]:
# TODO fortify data?
# you could use slicing here for large images
branch_array = zarr_utils.fortify(branch_data[0])

In [ ]:
# get branch data
branch_props = LabelPropsUtils(task_dir)\
    .label_props_view(value_name = 'stroma.branch')\
    .view_cols(['label', 'branch-type'])\
    .as_df()

In [ ]:
value_name = 'stroma.branch'
type_counts = dict()

# TODO there is probably a better way to do this
for i in list(branch_props['branch-type'].unique()):
    print(f'> get {i}')
    
    type_labels = branch_props.loc[branch_props['branch-type'] == i]['label'].values
    type_idx = np.isin(branch_array, type_labels)
    
    # get labels for branch type
    type_data = labels_data[0] * type_idx
    
    # get value counts for labels
    type_counts[f'{value_name}_type_{i}'] = np.unique(type_data, return_counts = True)

In [ ]:
# convert counts to dataframe
type_count_dfs = [
    pd.DataFrame(np.stack(x).T, columns = ['label', i]) for i, x in type_counts.items()]

In [ ]:
# https://www.statology.org/pandas-merge-multiple-dataframes/
from functools import reduce

# merge
type_count_df = reduce(lambda left,right: pd.merge(
    left, right, on = ['label'], how = 'outer'), type_count_dfs)

In [ ]:
# get labels
labels_ids = LabelPropsUtils(task_dir)\
    .label_props_view(value_name = 'default')\
    .view_label_col()\
    .values_obs()

In [ ]:
# merge counts to labels
merged_branching = labels_ids.join(type_count_df.set_index('label'), on = 'label')

In [ ]:
merged_branching.fillna(0, inplace = True)

In [ ]:
# convert to anndata spatial
label_view = LabelPropsUtils(task_dir).label_props_view(value_name = 'stroma.branch')

In [ ]:
label_view.adata.X[:, label_view.adata.var_names.get_loc('image-coord-dst-0')]

In [ ]:
dim_utils.im_scale(['z', 'x', 'y'])

In [ ]:
label_view.adata.X[:, label_view.adata.var_names.get_loc('coord-dst-0')]

In [ ]:
label_view.adata.var_names

In [ ]:
label_view.close()

In [ ]:
# locations are ZYX
min_pos_idx = [label_view.adata.var_names.get_loc(i) for i in label_view.adata.var_names if i.startswith('image-coord-src-')]
max_pos_idx = [label_view.adata.var_names.get_loc(i) for i in label_view.adata.var_names if i.startswith('image-coord-dst-')]

In [ ]:
min_pos_idx

In [ ]:
# create positions
# locations are ZYX
min_pos_idx = [label_view.adata.var_names.get_loc(i)
    for i in label_view.adata.var_names if i.startswith('image-coord-src-')]
max_pos_idx = [label_view.adata.var_names.get_loc(i)
    for i in label_view.adata.var_names if i.startswith('image-coord-dst-')]

In [ ]:
# get centre for coords
# https://stackoverflow.com/a/18461943
label_view.adata.obsm = {
    'spatial': np.mean(
          np.array([label_view.adata.X[:, min_pos_idx] * dim_utils.im_scale(['z', 'x', 'y']),
                    label_view.adata.X[:, max_pos_idx] * dim_utils.im_scale(['z', 'x', 'y'])]),
          axis=0)}

# create column identifier
label_view.adata.uns = {
  'spatial_cols': np.array(['centroid_z', 'centroid_y', 'centroid_x']),
  # 'spatial_neighbors': {
  #     'connectivities_key': 'spatial_connectivities',
  #     'distances_key': 'spatial_distances'
  # }
}

In [ ]:
label_view.adata.obsm['spatial']

In [ ]:
label_view.save()

In [ ]:
adata.uns = {
    'spatial_cols': np.array(['centroid_z', 'centroid_y', 'centroid_x'])
}

In [ ]:
# get connections
label_view.save()

In [ ]:
label_view = LabelPropsUtils(task_dir).label_props_view(value_name = 'default.sq')
spatial_adata = label_view.as_adata()
label_view.close()

In [ ]:
spatial_adata.uns['spatial_neighbors']

In [ ]:
spatial_adata

In [ ]:
spatial_adata.uns['spatial_cols']

In [ ]:
spatial_adata.obsp['spatial_connectivities'].toarray().max()

In [ ]:
# create dict
merged_dict = {
    i: merged_branching[i] for i in merged_branching.columns[merged_branching.columns != 'label']}

# add to obs and save
LabelPropsUtils(task_dir)\
    .label_props_view(value_name = 'default')\
    .add_obs(merged_dict)\
    .save()

In [ ]:
im_data[0].shape

In [ ]:
# analyse stroma network
stroma_data, _ = zarr_utils.open_labels_as_zarr(
    '/Volumes/Analysis_SSD/Dominik/cecelia/projects/wEsnzt/ANALYSIS/1/n22foC/labels/stroma.zarr')

In [ ]:
from skimage.morphology import skeletonize
import skan

In [ ]:
binary_skeleton = skeletonize(
    (np.squeeze(stroma_data[0]) > 0).astype(np.uint8))
skeleton = skan.Skeleton(binary_skeleton)

In [ ]:
all_paths = [
    skeleton.path_coordinates(i) for i in range(skeleton.n_paths)]

In [ ]:
all_paths

In [ ]:
paths_table.columns

In [ ]:
paths_table = skan.summarize(skeleton)

In [ ]:
paths_table['label'] = np.arange(skeleton.n_paths)

In [ ]:
paths_table['random-path-id'] = np.random.default_rng().permutation(skeleton.n_paths)

In [ ]:
paths_table.to_csv('/Volumes/USER_data/Dominik/Experiments/IBEX/CECELIA/stroma/n22foC_branching.csv')

In [ ]:
# save table
from py.label_props_utils import LabelPropsUtils

task_dir = '/Volumes/USER_data/Dominik/Experiments/IBEX/CECELIA/stroma/'
branching_name = 'stroma_branching'

LabelPropsUtils(task_dir, cfg.value_dir(branching_name, 'labelProps'))\
    .label_props(
      paths_table,
      save = True,
      obs_cols = [
        'skeleton-id', 'node-id-src', 'node-id-dst', 'branch-type'
        ]
      )

In [ ]:
# save labels
zarr_utils.create_multiscales(
    skimage.morphology.dilation(np.asarray(skeleton), skimage.morphology.ball(1)),
    '/Volumes/USER_data/Dominik/Experiments/IBEX/CECELIA/stroma/skeleton.zarr',
    dim_utils = napari_utils.dim_utils,
    nscales = len(napari_utils.im_data),
    keyword = 'labels',
    ignore_channel = True,
    reference_zarr = stroma_data[0]
)

In [ ]:
os.path.join(branching_name, 'data')

In [ ]:
import skimage.morphology

In [ ]:
skeleton_labels = skimage.morphology.dilation(np.asarray(skeleton), skimage.morphology.ball(1))

In [ ]:
skeleton_path = os.path.join(task_dir, 'labels/stroma.branch.zarr')

In [ ]:
nscales = len(stroma_data)

In [ ]:
import zarr

In [ ]:
skeleton_store = zarr.open(
    skeleton_path,
    mode = 'w',
    shape = stroma_data[0].shape,
    chunks = stroma_data[0].chunks,
    dtype = np.uint32)

In [ ]:
skeleton_store[:] = np.expand_dims(skeleton_labels, axis = 0)

In [ ]:
multiscales_file_path = skeleton_path + '.multiscales'

In [ ]:
napari_utils.dim_utils.dim_idx('X', ignore_channel = True)

In [ ]:
skeleton_store.shape

In [ ]:
zarr_utils.create_multiscales(
    skeleton_store, multiscales_file_path,
    dim_utils = napari_utils.dim_utils,
    nscales = nscales,
    keyword = 'labels',
    ignore_channel = True
)

In [ ]:
skeleton_layer = napari_utils.viewer.add_labels(
    np.expand_dims(skeleton_labels, axis = napari_utils.dim_utils.dim_idx('T')),
    properties = paths_table,
    opacity = 1,
    name = 'branching',
    scale = napari_utils.im_scale
)

In [ ]:
# get branching distance
#percentile = 99.8
percentile = 100
chnl_scale = 255
#branching_values = paths_table['branch-distance'].values
branching_values = paths_table['branch-type'].values
label_ids = paths_table['path-id'].values

# get max value from intensity
max_chnl_val = np.percentile(branching_values, percentile)

# how to select a map.. ?
labels_cm = plt.cm.viridis(np.linspace(0, 1, num = chnl_scale))
# labels_cm = plt.cm.bwr(np.linspace(0, 1, num = chnl_scale))

# insert background
labels_cm = np.insert(labels_cm, 0, 0, axis = 0)

layer_chnl_colours = np.array(branching_values/max_chnl_val * chnl_scale - 1).astype(np.int)
layer_chnl_colours[layer_chnl_colours >= chnl_scale] = chnl_scale - 1
layer_chnl_colours[layer_chnl_colours <= 0] = 1
layer_chnl_colours = list(layer_chnl_colours)

# insert background
layer_chnl_colours.insert(0, 0)

# convert to dict
skeleton_layer.color = {x: labels_cm[layer_chnl_colours[i]] for i, x in enumerate(label_ids)}

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
skeleton_labels.shape

In [ ]:
stroma_slices = (slice(None), slice(100, 1100, 1), slice(100, 1100, 1))

In [ ]:
# TODO this is not good
rgb = np.dstack(tuple([
    np.amax(skeleton_labels[stroma_slices] * np.isin(
    skeleton_labels[stroma_slices], paths_table.loc[paths_table['branch-type'] == x]['path-id'].values),
            axis = 0)
    for x in range(0, 3)
]))

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(30, 30))

axes.imshow(rgb)

plt.tight_layout()
plt.show()